In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pathlib
import tensorflow as tf

In [3]:
DATASET_LOCATION = '/content/drive/MyDrive/ai-playground/ra-text-classification-spectrogram/images/isot-4000-bert'
data_dir = pathlib.Path(DATASET_LOCATION)

In [4]:
TRAIN_FRACTION = 0.8
batch_size = 32 # default batch size of data
img_height = 224 # default height to which image is resized
img_width = 224 # default width to which image is resized
seed = 2022 # random seed for shuffling and transformations

In [5]:
ds_train = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split = 1 - TRAIN_FRACTION,
  subset = "training",
  seed = seed,
  image_size = (img_height, img_width),
  batch_size = batch_size)

Found 2332 files belonging to 2 classes.
Using 1866 files for training.


In [6]:
ds_val = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split = 1 - TRAIN_FRACTION,
  subset = "validation",
  seed = seed,
  image_size = (img_height, img_width),
  batch_size = batch_size)

Found 2332 files belonging to 2 classes.
Using 466 files for validation.


In [7]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = ds_train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = ds_val.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
class_names = ds_train.class_names
class_names

['fake', 'true']

In [9]:
VGG16_MODEL = tf.keras.applications.VGG16(
    input_shape = (img_height, img_width, 3),
    include_top = False,
    weights = 'imagenet'
    )

# ------ VGG 19 -------
# base_model_sub = tf.keras.applications.vgg19.VGG19(
#     input_shape = (img_height, img_width, 3),
#     include_top = False,
#     weights = 'imagenet'
#     )

In [10]:
VGG16_MODEL.trainable = False

# ------ VGG 19 -------
# base_model_sub.trainable = False

In [15]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
# prediction_layer = tf.keras.layers.Dense(2, activation='softmax') # binary is better with sigmoid
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')

In [16]:
model = tf.keras.Sequential([
  VGG16_MODEL,
  # base_model_sub, VGG 19
  global_average_layer,
  prediction_layer
])

In [17]:
# model.compile(
#   optimizer = 'adam',
#   loss=tf.keras.losses.sparse_categorical_crossentropy,
#   learning_rate=0.01)

model.compile(optimizer='adam', loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
history = model.fit(
    ds_train,
    epochs = 100,
    validation_data=ds_val,
    verbose = 1)

Epoch 1/100
59/59 [==============================] - 26s 314ms/step - loss: 1.8235 - accuracy: 0.5032 - val_loss: 0.7961 - val_accuracy: 0.5064
Epoch 2/100
59/59 [==============================] - 15s 235ms/step - loss: 0.7387 - accuracy: 0.5445 - val_loss: 0.7292 - val_accuracy: 0.5536
Epoch 3/100
59/59 [==============================] - 16s 249ms/step - loss: 0.6809 - accuracy: 0.5965 - val_loss: 0.6856 - val_accuracy: 0.6052
Epoch 4/100
59/59 [==============================] - 15s 245ms/step - loss: 0.6592 - accuracy: 0.6083 - val_loss: 0.6561 - val_accuracy: 0.6373
Epoch 5/100
59/59 [==============================] - 15s 236ms/step - loss: 0.6188 - accuracy: 0.6704 - val_loss: 0.6397 - val_accuracy: 0.6609
Epoch 6/100
59/59 [==============================] - 15s 234ms/step - loss: 0.5918 - accuracy: 0.7117 - val_loss: 0.6113 - val_accuracy: 0.6910
Epoch 7/100
59/59 [==============================] - 15s 243ms/step - loss: 0.5889 - accuracy: 0.7031 - val_loss: 0.6075 - val_accuracy:

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('vgg 16 model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()